In [1]:
import os
import numpy as np
import time
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

In [2]:
from keras.datasets import mnist

# load and normalize data
#(X_train, y_train), (X_test, y_test) =  tf.keras.datasets.mnist.load_data()
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()


X_train = tf.reshape(X_train, (X_train.shape[0],-1))/255
X_test = tf.reshape(X_test, (X_test.shape[0],-1))/255

#reserve the last 10000 training examples for validation
X_train, X_val = X_train[:-10000], X_train[-10000:]
y_train, y_val = y_train[:-10000], y_train[-10000:]

print("train_size:", X_train.shape)
print("val_size:", X_val.shape)
print("test_size:", X_test.shape)
print("train_output_size", y_train.shape)
print("val_output_size:", y_val.shape)
print("test_output_size", y_test.shape)

size_input = X_train.shape[1]
size_output = len(set(y_train))
size_hidden1 = 128
size_hidden2 = 128
size_hidden3 = 128

train_size: (50000, 784)
val_size: (10000, 784)
test_size: (10000, 784)
train_output_size (50000,)
val_output_size: (10000,)
test_output_size (10000,)


# MLP with BN -- BN(g(f(x)))

In [3]:
# Define class to build mlp model
class MLP(object):
    def __init__(self, size_input, size_hidden1, size_hidden2, size_output, device=None,\
                 regularizer=None, R_lambda = 1e-4, drop_prob=0):
        """
        size_input: int, size of input layer
        size_hidden: int, size of hidden layer
        size_output: int, size of output layer
        device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
        regularizer: str or None
        R_lambda: the parameter for regularizer
        drop_prob: 0 to 1
        """
        self.size_input, self.size_hidden1, self.size_hidden2, self.size_hidden3, self.size_output, self.device =\
        size_input, size_hidden1, size_hidden2, size_hidden3, size_output, device
        self.regularizer, self.R_lambda, self.drop_prob = regularizer, R_lambda, drop_prob
        
        
        # Initialize BN scale parameter in the input layer
        self.gamma0 = tf.Variable(tf.ones([1, self.size_input]))
        # Initialize BN shift parameter in the input layer
        self.beta0 = tf.Variable(tf.zeros([1, self.size_input]))
        # The moving mean and variance in the input layer
        self.moving_mean0 = tf.Variable(tf.zeros([1, self.size_input]))
        self.moving_var0 = tf.Variable(tf.ones([1, self.size_input]))
        
        # Initialize weights between input mapping and a layer g(f(x)) = layer
        self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden1],stddev=0.1)) # Xavier(Fan-in fan-out) and Orthogonal
        # Initialize biases for hidden layer
        self.b1 = tf.Variable(tf.zeros([1, self.size_hidden1])) # 0 or constant(0.01)
        
        # Initialize BN scale parameter in the hidden layer 1
        self.gamma1 = tf.Variable(tf.ones([1, self.size_hidden1]))
        #Initialize BN shift parameter in the hidden layer 1
        self.beta1 = tf.Variable(tf.zeros([1, self.size_hidden1]))
        # The moving mean and variance in the input layer
        self.moving_mean1 = tf.Variable(tf.zeros([1, self.size_hidden1]))
        self.moving_var1 = tf.Variable(tf.ones([1, self.size_hidden1]))

        # Initialize weights between input layer and 1st hidden layer
        self.W2 = tf.Variable(tf.random.normal([self.size_hidden1, self.size_hidden2],stddev=0.1))
        # Initialize biases for hidden layer
        self.b2 = tf.Variable(tf.zeros([1, self.size_hidden2]))
        
        # Initialize BN scale parameter in the hidden layer 2
        self.gamma2 = tf.Variable(tf.ones([1, self.size_hidden2]))
        # Initialize BN shift parameter in the hidden layer 2
        self.beta2 = tf.Variable(tf.zeros([1, self.size_hidden2]))
        # The moving mean and variance in the input layer
        self.moving_mean2 = tf.Variable(tf.zeros([1, self.size_hidden2]))
        self.moving_var2 = tf.Variable(tf.ones([1, self.size_hidden2]))

        # Initialize weights between 1st hidden layer and 2nd hidden layer
        self.W3 = tf.Variable(tf.random.normal([self.size_hidden2, self.size_hidden3],stddev=0.1))
        # Initialize biases for hidden layer
        self.b3 = tf.Variable(tf.zeros([1, self.size_hidden3]))
        
        # Initialize BN scale parameter in the hidden layer 3
        self.gamma3 = tf.Variable(tf.ones([1, self.size_hidden3]))
        #Initialize BN shift parameter in the hidden layer 3
        self.beta3 = tf.Variable(tf.zeros([1, self.size_hidden3]))
        # The moving mean and variance in the input layer
        self.moving_mean3 = tf.Variable(tf.zeros([1, self.size_hidden3]))
        self.moving_var3 = tf.Variable(tf.ones([1, self.size_hidden3]))

         # Initialize weights between 2nd hidden layer and output layer
        self.W4 = tf.Variable(tf.random.normal([self.size_hidden3, self.size_output],stddev=0.1))
        # Initialize biases for output layer
        self.b4 = tf.Variable(tf.zeros([1, self.size_output]))

        # Define variables to be updated during backpropagation
        self.variables = [self.W1, self.W2, self.W3, self.W4, \
                          self.b1, self.b2, self.b3, self.b4, \
                          self.gamma0, self.gamma1, self.gamma2, self.gamma3, \
                          self.beta0, self.beta1, self.beta2, self.beta3]
        
       
    def forward(self, X, training):
        """
        forward pass
        X: Tensor, inputs
        training: bool. True if it's training and False if it's predicting
        """
        def compute_output(X, training):
            BN_eps = 1e-5
            # Cast X to float32
            X_tf = tf.cast(X, dtype=tf.float32)
            
#             #set the dropout prob
#             prob = self.drop_prob
            
            # BN in input layer
            if training:
                # compute the sample_mean and sample_var for current batch
                sample_mean0 = tf.math.reduce_mean(X_tf, axis=0)
                sample_var0 = tf.math.reduce_variance(X_tf, axis=0)
                X_tf = self.batch_norm(X_tf, self.gamma0, self.beta0, sample_mean0, sample_var0, BN_eps)
                
                # update the self.moving_mean0 and self.moving_var0
                self.assign_moving_average(self.moving_mean0, sample_mean0)
                self.assign_moving_average(self.moving_var0, sample_var0)
            else:
                # using the updated moving_mean and moving_var for batch normalization
                X_tf = self.batch_norm(X_tf, self.gamma0, self.beta0, self.moving_mean0, self.moving_var0, BN_eps)

            # Remember to normalize your dataset before moving forward
            # Compute values in hidden layer 1
            what1 = tf.matmul(X_tf, self.W1) + self.b1

            # activation in hiddent layer 1
            hhat1 = tf.nn.relu(what1)
            
            # BN in hidden layer 1
            if training:
                # compute the sample_mean and sample_var for current batch
                sample_mean1 = tf.math.reduce_mean(hhat1, axis=0)
                sample_var1 = tf.math.reduce_variance(hhat1, axis=0)
                hhat1 = self.batch_norm(hhat1, self.gamma1, self.beta1, sample_mean1, sample_var1, BN_eps)
                
                # update the self.moving_mean1 and self.moving_var1
                self.assign_moving_average(self.moving_mean1, sample_mean1)
                self.assign_moving_average(self.moving_var1, sample_var1)
            else:
                # using the updated moving_mean and moving_var for batch normalization
                hhat1 = self.batch_norm(hhat1, self.gamma1, self.beta1, self.moving_mean1, self.moving_var1, BN_eps)

            # Compute values in hidden layer 2
            what2 = tf.matmul(hhat1, self.W2) + self.b2
            
            # activation in hiddent layer 2
            hhat2 = tf.nn.relu(what2)
            
            # BN in hidden layer 2
            if training:
                # compute the sample_mean and sample_var for current batch
                sample_mean2 = tf.math.reduce_mean(hhat2, axis=0)
                sample_var2 = tf.math.reduce_variance(hhat2, axis=0)
                hhat2 = self.batch_norm(hhat2, self.gamma2, self.beta2, sample_mean2, sample_var2, BN_eps)
                
                # update the self.moving_mean2 and self.moving_var2
                self.assign_moving_average(self.moving_mean2, sample_mean2)
                self.assign_moving_average(self.moving_var2, sample_var2)
            else:
                # using the updated moving_mean and moving_var for batch normalization
                hhat2 = self.batch_norm(hhat2, self.gamma2, self.beta2, self.moving_mean2, self.moving_var2, BN_eps)
            
            # Compute values in hidden layer 3
            what3 = tf.matmul(hhat2, self.W3) + self.b3
            
            # activation in hiddent layer 3
            hhat3 = tf.nn.relu(what3)
            
            # BN in hidden layer 3
            if training:
                # compute the sample_mean and sample_var for current batch
                sample_mean3 = tf.math.reduce_mean(hhat3, axis=0)
                sample_var3 = tf.math.reduce_variance(hhat3, axis=0)
                hhat3 = self.batch_norm(hhat3, self.gamma3, self.beta3, sample_mean3, sample_var3, BN_eps)
                
                # update the self.moving_mean3 and self.moving_var3
                self.assign_moving_average(self.moving_mean3, sample_mean3)
                self.assign_moving_average(self.moving_var3, sample_var3)
            else:
                # using the updated moving_mean and moving_var for batch normalization
                hhat3 = self.batch_norm(hhat3, self.gamma3, self.beta3, self.moving_mean3, self.moving_var3, BN_eps)
            
            # Compute output
            what4 = tf.matmul(hhat3, self.W4) + self.b4
            output = tf.nn.softmax(what4)

            return output
        
        if self.device is not None:
            with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
                self.y = compute_output(X, training)
        else:
            self.y = compute_output(X, training)

        return self.y
    
    def assign_moving_average(self, variable, value):
        momentum = 0.99
        delta = variable * momentum + value * (1 - momentum)
        return variable.assign(delta)
    
    def loss(self, y_pred, y_true):
        '''
        y_pred - Tensor of shape (batch_size, size_output)
        y_true - Tensor of shape (batch_size, size_output)
        '''  
        #cross entropy loss for classifation mission
        return tf.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits = False)
        #return tf.reduce_sum(-tf.math.log(tf.boolean_mask(y_pred, tf.one_hot(y_true, depth=y_pred.shape[-1]))))/y_pred.shape[0]
    
    def batch_norm(self, X, gamma, beta, moving_mean, moving_var, eps):
        # Compute reciprocal of square root of the moving variance elementwise
        inv = tf.cast(tf.math.rsqrt(moving_var + eps), X.dtype)
        # Scale and shift
        inv *= gamma
        Y = X * inv + (beta - moving_mean * inv)
        return Y

    def backward(self, X_train, y_train, hyperparams, method='sgd'):
        """
        backward pass
        """
        with tf.GradientTape() as tape:
            predicted = self.forward(X_train, training = True)
            current_loss = self.loss(predicted, y_train)
            
            num_layer = 3
            if not self.regularizer:
                current_loss = self.loss(predicted, y_train)
            elif self.regularizer == 'l2':
                #flatten shape
                w = tf.concat([tf.reshape(w,[-1]) for w in self.variables[:num_layer]],0)
                current_loss  += self.R_lambda * tf.nn.l2_loss(w)
            elif self.regularizer == 'l1':
                #flatten shape
                w = tf.concat([tf.reshape(w,[-1]) for w in self.variables[:num_layer]],0)
                current_loss  += self.R_lambda * tf.nn.l1_loss(w)
            
        grads = tape.gradient(current_loss, self.variables)
        
        if method == 'sgd':
            optimizer = tf.keras.optimizers.SGD(learning_rate = hyperparams['lr'])
            optimizer.apply_gradients(zip(grads, self.variables))
    

    def accuracy(self,y_pred, y_true):
        """
        compute the correct num
        y_pred: the probability distribution [[...]] or the predicted label [...]
        y_true: the 1-D true label
        """
        #detect if y_pred is a probability distribution 
        if len(y_pred.shape) > 1 and y_pred.shape[1] > 1:
            y_pred = tf.argmax(y_pred, axis=1)
            
        cmp = tf.cast(y_pred, y_true.dtype) == y_true
        
        return float(tf.reduce_sum(tf.cast(cmp, tf.int32)))
    
#     def dropout_layer(self,X, dropout):
#         assert 0 <= dropout <= 1
#         # In this case, all elements are dropped out
#         if dropout == 1:
#             return tf.zeros_like(X)
#         # In this case, all elements are kept
#         if dropout == 0:
#             return X
#         mask = tf.random.uniform(
#             shape=tf.shape(X), minval=0, maxval=1) < 1 - dropout
#         return tf.cast(mask, dtype=tf.float32) * X / (1.0 - dropout)


# Train Model with SGD from keras

In [6]:
#save the model for tuning
import pickle
def save_object(obj, filename):
    # Overwrites any existing file.
    with open(filename, 'wb') as file:  
        pickle.dump(obj, file, pickle.HIGHEST_PROTOCOL)
        
def load_object(filename):
    # Open the file in binary mode
    with open(filename, 'rb') as file:  
        # Call load method to deserialze
        return pickle.load(file)

# Set number of simulations and epochs
NUM_SIM = 3
NUM_EPOCHS = 100

#set the train_record
train_loss_record = [[] for _ in range(NUM_SIM)]
train_acc_record = [[] for _ in range(NUM_SIM)]
val_loss_record = [[] for _ in range(NUM_SIM)]
val_acc_record = [[] for _ in range(NUM_SIM)]
test_loss_record = [[] for _ in range(NUM_SIM)]
test_acc_record = [[] for _ in range(NUM_SIM)]

for num_sim in range(NUM_SIM):
    np.random.seed(num_sim)
    tf.random.set_seed(num_sim)
    '''
    Initialize model using GPU or load an exitsing MLP
    '''
    mlp_DIY = MLP(size_input, size_hidden1, size_hidden2, size_output, device='GPU',\
                regularizer=None, R_lambda = 1e-4, drop_prob=0.)
#     mlp_DIY = load_object('mlp_DIY.pkl')

    time_start = time.time()
    hyperparams = {'t':1, 'lr':1e-4}

    for epoch in range(NUM_EPOCHS):
        train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(25, seed=epoch*num_sim).batch(128)

        for inputs, outputs in train_ds:
            preds = mlp_DIY.forward(inputs, training = True)

            #use SGD to train the model
            mlp_DIY.backward(inputs, outputs, hyperparams,'sgd')
            hyperparams['t'] += 1
        
        if (epoch + 1)%10 == 0:
            #compute the result for the current epoch
            logits = mlp_DIY.forward(X_train, training = False)
            train_loss = np.sum(mlp_DIY.loss(logits, y_train))/len(y_train)
            train_acc = mlp_DIY.accuracy(logits,y_train)/len(y_train)
            train_loss_record[num_sim].append(train_loss)
            train_acc_record[num_sim].append(train_acc)

            logits = mlp_DIY.forward(X_val, training = False)
            val_loss = np.sum(mlp_DIY.loss(logits, y_val))/len(y_val)
            val_acc = mlp_DIY.accuracy(logits,y_val)/len(y_val)
            val_loss_record[num_sim].append(val_loss)
            val_acc_record[num_sim].append(val_acc)
            
            logits = mlp_DIY.forward(X_test, training = False)
            test_loss = np.sum(mlp_DIY.loss(logits, y_test))/len(y_test)
            test_acc = mlp_DIY.accuracy(logits,y_test)/len(y_test)
            test_loss_record[num_sim].append(test_loss)
            test_acc_record[num_sim].append(test_acc)
            
            print('Number of Simulation = {} - Number of Epoch = {}'.format(num_sim+1, epoch + 1))
            print('Train loss:= {:.4f} - Val loss: {:.4f} - Test loss: {:.4f} - Train acc:= {:.2%} - Val acc:= {:.2%} - Test acc:= {:.2%}'\
                  .format(train_loss, val_loss, test_loss, train_acc, val_acc, test_acc))
            
    time_taken = time.time() - time_start 
    print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))
#save_object(mlp_DIY,'mlp_DIY.pkl')

Number of Simulation = 1 - Number of Epoch = 10
Train loss:= 0.3386 - Val loss: 0.3887 - Test loss: 0.4193 - Train acc:= 87.95% - Val acc:= 86.08% - Test acc:= 85.43%
Number of Simulation = 1 - Number of Epoch = 20
Train loss:= 0.2734 - Val loss: 0.3663 - Test loss: 0.3969 - Train acc:= 90.28% - Val acc:= 86.77% - Test acc:= 86.46%
Number of Simulation = 1 - Number of Epoch = 30
Train loss:= 0.2307 - Val loss: 0.3720 - Test loss: 0.4037 - Train acc:= 91.85% - Val acc:= 87.17% - Test acc:= 86.77%
Number of Simulation = 1 - Number of Epoch = 40
Train loss:= 0.1971 - Val loss: 0.3943 - Test loss: 0.4268 - Train acc:= 93.22% - Val acc:= 87.04% - Test acc:= 86.70%
Number of Simulation = 1 - Number of Epoch = 50
Train loss:= 0.1701 - Val loss: 0.4266 - Test loss: 0.4619 - Train acc:= 94.20% - Val acc:= 86.84% - Test acc:= 86.64%
Number of Simulation = 1 - Number of Epoch = 60
Train loss:= 0.1498 - Val loss: 0.4670 - Test loss: 0.5097 - Train acc:= 94.98% - Val acc:= 86.73% - Test acc:= 86.49